In [6]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import math 
import numpy as np 
import matplotlib.pyplot as plt
import logging 
import cv2 as cv

datasets, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True) # Loads the data
train_dataset, test_dataset = datasets['train'], datasets['test'] # Splits the dataset 


In [7]:
class_name = metadata.features['label'].names # Prints the names of the labels
print(class_name)
print(metadata.features) # Prints the features


num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), #flattens the 3d image into one array of numbers
    tf.keras.layers.Dense(512, activation=tf.nn.relu), #creates 128 level of neural networks
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #softmax produces probability distribution 
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy']) 

BATCH_SIZE = 32 #Batch size is the number of iterations the neural network does before it is updated 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE) #Updates dataset with batch included
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
Number of training examples: 60000
Number of test examples:     10000


In [11]:
import os
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_test_examples/32), callbacks=[cp_callback])

model.save_weights("model weights")
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)
print('Loss: ', test_loss)

for test_images, test_labels in test_dataset.take(1): #predict one sample from the data
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

predictions.shape
print(np.argmax(predictions[0]))

Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.4219 - accuracy: 0.8452
Epoch 1: saving model to training_2\cp-0001.ckpt
313/313 [==============================] - 18s 54ms/step - loss: 0.4219 - accuracy: 0.8452
Epoch 2/10
312/313 [============================>.] - ETA: 0s - loss: 0.4380 - accuracy: 0.8458
Epoch 2: saving model to training_2\cp-0002.ckpt
313/313 [==============================] - 16s 50ms/step - loss: 0.4375 - accuracy: 0.8458
Epoch 3/10
312/313 [============================>.] - ETA: 0s - loss: 0.4362 - accuracy: 0.8406
Epoch 3: saving model to training_2\cp-0003.ckpt
313/313 [==============================] - 14s 46ms/step - loss: 0.4359 - accuracy: 0.8408
Epoch 4/10
313/313 [==============================] - ETA: 0s - loss: 0.4286 - accuracy: 0.8433
Epoch 4: saving model to training_2\cp-0004.ckpt
313/313 [==============================] - 15s 48ms/step - loss: 0.4286 - accuracy: 0.8433
Epoch 5/10
313/313 [==============================] - ET

In [9]:
model.save_weights("WEIGHTS")
